In [29]:
from datetime import datetime
from elasticsearch import Elasticsearch


es = Elasticsearch('127.0.0.1:9200')

In [30]:
def segmentation(partition):
    import os
    import re
    import jieba
    import jieba.analyse
    import jieba.posseg as pseg
    import codecs

    abspath = "C:/Users/fscut/Desktop/OA讨论"

    # # 结巴加载用户词典
    #userDict_path = os.path.join(abspath, "ITKeywords.txt")
    #jieba.load_userdict(userDict_path)
    #
    # # 停用词文本
    stopwords_path = os.path.join(abspath, "stopwords.txt")
    def get_stopwords_list():
    #     """返回stopwords列表"""
         stopwords_list = [i.strip() for i in codecs.open(stopwords_path, encoding = 'UTF-8').readlines()]
         return stopwords_list
    # # 所有的停用词列表
    stopwords_list = get_stopwords_list()



    # 分词
    def cut_sentence(sentence):
        """对切割之后的词语进行过滤，去除停用词，保留名词，英文和自定义词库中的词，长度大于2的词"""
        seg_list = pseg.lcut(sentence)
        seg_list = [i for i in seg_list if i.word not in stopwords_list]
        filtered_words_list = []
        for seg in seg_list:
            if len(seg.word) <= 1:
                continue
            elif seg.flag == "eng":
                if len(seg.word) <= 2:
                    continue
                else:
                    filtered_words_list.append(seg.word)
            elif seg.flag.startswith("n"):
                filtered_words_list.append(seg.word)
            elif seg.flag in ["eng"]:  # 是自定一个词语或者是英文单词
                filtered_words_list.append(seg.word)
        return filtered_words_list

    result = []
    for row in partition:
        #sentence = re.sub("<.*?>", "")  # 替换掉标签数据
        words = cut_sentence(row)
        result += words
    return result





In [31]:
import requests
import json
import re
from collections import Counter
import math



s = requests.Session()
url = 'http://oa.fscut.com/seeyon/ajax.do?d=4095&method=ajaxAction&managerName=bbsArticleManager'

cookie = 'JSESSIONID=6CA24E3B955E05BCCB4D5EB165CC14FE'

headers = {
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
            'cookie': cookie,
            'Referer': 'http://oa.fscut.com/seeyon/bbs.do?method=bbsIndex&portalId=-3489196460133432033&_resourceCode=F05_bbsIndexAccount'
        }
formdata = {
                "managerMethod": "findListData",
                "arguments":'[{"pageSize":"3000","pageNo":1,"spaceType":"","spaceId":"","boardId":"","listType":"latestArticle","condition":"","textfield1":"","textfield2":""}]'
           }


r = s.post(url, data = formdata, headers = headers)


page_dict = json.loads(r.text)


page_dict = page_dict['list']

def get_oa_date(id):
    for page_info in page_dict:
        if page_info['id'] == id:
            return [page_info['issueTime'], page_info['issueName']]




In [32]:
from atlassian import Jira
from atlassian import Confluence
from atlassian import Bitbucket
from atlassian import ServiceDesk

confluence = Confluence(
    url='http://docs.fscut.com',
    username='jiangchenglin',
    password='jcl565600')


def get_confluence_date(id):
    page_info = confluence.get_page_by_id(id, expand = 'space,body.view,version,container')
    return page_info['version']['when']



In [37]:
get_confluence_date('53489068')

'2020-07-10T19:53:01.000+08:00'

In [33]:
import os
with open(r'C:\Users\fscut\Desktop\OA讨论\id.txt', "r", encoding="utf-8") as f:
    ids = f.read().split()
    
base_dir = r'C:\Users\fscut\Desktop\OA讨论\reports'

corpus = []

for id in ids[1:]:
    print(id)
    path = os.path.join(base_dir, id + '.txt')
    with open(path, "r", encoding='utf-8') as f:
        title = f.readline().split()
        article = f.read()

    corpus.append(' '. join(segmentation(article.split())))
    

In [34]:
import os  
import sys  
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

 
    
    


vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值  
tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus))#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵  
word=vectorizer.get_feature_names()#获取词袋模型中的所有词语  
weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重  
#for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重  
    #print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------")
    #for j in range(len(word)):
        #if weight[i][j] > 0.1:
            #print (word[j],weight[i][j])

In [39]:
import os
import numpy as np

es_id = 0
for id in ids[1:] :
    print(id)
    path = os.path.join(base_dir, id + '.txt')
    with open(path, "r", encoding='utf-8') as f:
        title = f.readline().split()
        article = f.read()
    doc  = {}
    doc['id'] = id
    doc['title'] = ''.join(title)
    doc['text'] = article
    if len(id) > 14:
        doc['url'] = 'http://oa.fscut.com/seeyon/bbs.do?method=bbsView&articleId=' + id + '&spaceType=&spaceId='
        doc['issue_date'] = get_oa_date(id)[0][0:10]
        doc['author'] = get_oa_date(id)[1]
    else :
        doc['url'] = 'http://docs.fscut.com/pages/viewpage.action?pageId=' + id
        doc['issue_date'] = get_confluence_date(id)[0:10]
    words = []
    for pos in np.argsort(-weight[es_id, :])[0:10]:
        words.append(word[pos])
    doc['keywords'] = words
    print( doc['issue_date'])
    res = es.index(index="articles", id=es_id, body=doc)
    es_id += 1

-2696153348591432670
2020-07-16
4971304444312437621
2020-07-06
7911651041792706761
2020-06-30
4020014394070878549
2020-06-23
5192815444388956262
2020-06-20
-8043725623723144437
2020-06-14
4037647829680213211
2020-06-13
-3038039597326710076
2020-06-13
590744441766298457
2020-06-13
-3779655148012156017
2020-06-13
-4317764853001336623
2020-06-12
7484886418487070499
2020-06-11
7995835923654198967
2020-06-09
3544069372531722139
2020-06-05
-6983523811562795632
2020-05-28
-6291339583005637840
2020-05-27
-1552387664427073490
2020-05-25
-8735969838760771545
2020-05-25
-8084761168016031513
2020-05-22
-4921072683783075538
2020-05-22
-5205703266422207881
2020-05-14
4144250447844099711
2020-05-08
5420986428131264051
2020-05-08
-3488330644995683278
2020-04-29
-7848049429647505309
2020-04-21
-838247824462888954
2020-04-20
1197943394158202027
2020-04-18
1190746523259849961
2020-04-17
-3671681299642496136
2020-04-16
-2267868653231323480
2020-04-16
-1069296097024784789
2020-04-11
-1143728781014643844
20

-5601922924688357318
2018-11-15
134628890682281822
2018-11-13
2819213311760157311
2018-11-13
3970228323808004847
2018-11-12
-8708858398093719184
2018-11-12
-2619963209687546157
2018-11-09
-5822857484634405798
2018-11-07
8697969420078825681
2018-11-07
-8437728103945451957
2018-11-07
-1655156007848948189
2018-11-06
4411542419170217185
2018-11-06
2602042250456551320
2018-11-06
2299722493025665738
2018-11-06
7216151743748984646
2018-11-06
-8695650567095959333
2018-11-06
8889043552845385103
2018-11-05
-229815329079802389
2018-11-05
4451796209947003083
2018-11-05
-6908755649916681299
2018-11-05
4189021085303390951
2018-11-01
1884778977636536280
2018-11-01
-5074434489710066316
2018-10-29
-930856847976608245
2018-10-25
-266726699980339204
2018-10-22
5616604812553636952
2018-10-22
1076590823751106704
2018-10-22
7475487958153838884
2018-10-22
3298787347531383811
2018-10-19
4755357030174379066
2018-10-18
534597782743697499
2018-10-18
-1788635411582661777
2018-10-18
-3178625432161861295
2018-10-15

4315293752462166254
2018-07-02
3154290505141013297
2018-07-02
-2853959849010773654
2018-07-02
1174210538026582551
2018-07-02
4537843010748105329
2018-07-02
-2267675050789876774
2018-07-02
-3803445211041020467
2018-07-02
-2595549382620001450
2018-07-02
5731683264228639483
2018-07-02
5409939879359104762
2018-07-02
8671339196365030088
2018-07-01
-4199441376976289203
2018-07-01
-3155795192991035603
2018-06-30
-7078436544292959985
2018-06-30
-7830576447732953833
2018-06-30
7277809959000689968
2018-06-30
-5385966744294833023
2018-06-30
-5130101429543877983
2018-06-30
-891542052069488724
2018-06-30
-1451799039980057014
2018-06-30
-1233628175580051599
2018-06-30
-3484668836658397067
2018-06-30
3850224701050668778
2018-06-29
1853410916384983707
2018-06-29
1569380418504642184
2018-06-29
-7867928068326339009
2018-06-29
-8641025532009953317
2018-06-29
6074889714658855165
2018-06-29
6610778699634920963
2018-06-29
-3773185368778390666
2018-06-23
-1898983097172408507
2018-06-22
-6905415203768414478
2

6769919971237185476
2018-02-28
-2048574134115284475
2018-02-28
-679684918765336544
2018-02-27
1394823099027705699
2018-02-27
7621354038351841279
2018-02-24
4990041286466684355
2018-02-11
6333850598789106282
2018-02-09
-4481150200109119887
2018-02-09
971506229518877775
2018-02-09
8560172685534971622
2018-02-09
-3751573175657608491
2018-02-08
6534696467073286473
2018-02-08
8476816544349176619
2018-02-07
-3793907166840403559
2018-02-06
5699231784437697162
2018-02-06
321400363630020353
2018-02-05
-7863537292483146977
2018-02-05
-474901761876016284
2018-02-05
5568502361650381375
2018-02-05
3096894687380523853
2018-02-05
-704173985057978221
2018-02-05
-4505268355187259800
2018-02-05
2527020070133909944
2018-02-03
4620432833989397066
2018-02-03
1854343500409425886
2018-02-02
-7532410186194268250
2018-02-02
-3400602782818677722
2018-02-02
-4920654243860364222
2018-02-02
140888294215993215
2018-02-02
-8701503444052128869
2018-02-02
-4233995205082985594
2018-02-02
-4827089880830098651
2018-02-02

3507102026772990371
2017-11-30
-2714725550437874076
2017-11-30
-2003036306662145957
2017-11-30
-7894568942044952324
2017-11-30
-2609793709481179684
2017-11-30
3595153328400845707
2017-11-30
-2862660263522840673
2017-11-30
4699903652156427626
2017-11-30
-8886153613718960023
2017-11-30
-2700970036840676847
2017-11-30
-4858920622023753567
2017-11-30
500390960285303939
2017-11-30
-1287650022134559602
2017-11-30
1542734542113816911
2017-11-30
-174386125668728105
2017-11-30
-1953468549747619169
2017-11-30
4298936073429009797
2017-11-30
-3630033768925149602
2017-11-30
-5669166578881507405
2017-11-30
-7525755460575278790
2017-11-30
5217501934686326741
2017-11-30
431036132324167496
2017-11-30
-2510947101359127012
2017-11-30
2615115202914111034
2017-11-30
-2925290739654507943
2017-11-30
-3629251132716593234
2017-11-30
-5169707478686809173
2017-11-30
117007785103478564
2017-11-30
-1185485249881552843
2017-11-30
-8509176901160451632
2017-11-30
51154908592360690
2017-11-29
-8671207528579055480
2017

-8280611919509896106
2017-09-05
2429488293869015004
2017-09-05
-6569837550473231097
2017-09-05
7733082259623724442
2017-09-04
-2865953504742061841
2017-09-04
674236670265083513
2017-09-04
1530507992184931767
2017-09-04
8687055006335911549
2017-09-04
-5935462370003630897
2017-09-04
848017951671730607
2017-09-04
-7492520292209245108
2017-09-04
-7557703891136263820
2017-09-04
-2915409210399178426
2017-09-04
4839569257490761749
2017-09-04
8145189515674206744
2017-09-03
1255420473928600199
2017-09-02
-4510527737763508891
2017-09-02
-8272586138899364246
2017-09-02
940843208011697392
2017-09-02
-3350886412297548947
2017-09-02
8105348236255448700
2017-09-02
7268902600201946698
2017-09-02
9098951431965720655
2017-09-02
8647057130886743236
2017-09-02
-8775951620494176665
2017-09-02
4633705881454133651
2017-09-02
7841110116065167655
2017-09-02
3581858454861532225
2017-09-02
-7806778683795814770
2017-09-02
-4630002722045211597
2017-09-02
815868320117378158
2017-09-02
-6224977929035887905
2017-09-0

2017-06-29
569716981350354357
2017-06-29
4830017372201175067
2017-06-29
-5285045021591060445
2017-06-29
-2665098955168658300
2017-06-29
8549740746492300540
2017-06-29
5520761221609049216
2017-06-29
4892179837409968447
2017-06-29
232593121789430091
2017-06-29
2272737888654544832
2017-06-29
8300744931169683806
2017-06-28
-7591456593585420060
2017-06-28
-2367765867256000829
2017-06-28
-716629892347051178
2017-06-28
4155592103668564520
2017-06-28
-6105975450161821687
2017-06-28
-8915689789465933402
2017-06-28
2471207995671661708
2017-06-28
7147944113534944152
2017-06-28
-5351715581850664425
2017-06-27
-3714312211316389243
2017-06-27
4582519832414932045
2017-06-27
-544594012208477449
2017-06-24
-8095733698544450058
2017-06-24
1959978344182661206
2017-06-22
6786054176008916006
2017-06-21
2286293716902691582
2017-06-19
810714725404459068
2017-06-19
-640812452654593435
2017-06-17
4653812210456546483
2017-06-15
9040605594382191061
2017-06-09
7701101566689101038
2017-06-05
-6450918236188555239
2

53487495
2020-07-09
53487496
2020-07-09
53487497
2020-07-09
53487498
2020-07-09
53487499
2020-07-09
53487500
2020-07-09
53487502
2020-07-09
53487503
2020-07-09
53487504
2020-07-09
53487505
2020-07-09
53487506
2020-07-09
53487507
2020-07-09
53487508
2020-07-09
53487510
2020-07-09
53487511
2020-07-09
53487512
2020-07-09
53487513
2020-07-09
53487514
2020-07-09
53487515
2020-07-09
53487516
2020-07-09
53487517
2020-07-09
53487518
2020-07-09
53487519
2020-07-09
53487520
2020-07-09
53487521
2020-07-09
53487522
2020-07-09
53487523
2020-07-09
53487524
2020-07-09
53487525
2020-07-09
53487526
2020-07-09
53487527
2020-07-09
53487528
2020-07-09
53487529
2020-07-09
53487530
2020-07-09
53487531
2020-07-09
53487532
2020-07-09
53487533
2020-07-09
53487534
2020-07-09
53487535
2020-07-09
53487536
2020-07-09
53487537
2020-07-09
53487538
2020-07-09
53487539
2020-07-09
53487540
2020-07-09
53487541
2020-07-09
53487542
2020-07-09
53487888
2020-07-09
53487889
2020-07-09
53487890
2020-07-09
53487891
2020-07-09


53489164
2020-07-10
53489165
2020-07-10
53489166
2020-07-10
53489167
2020-07-10
53489168
2020-07-10
53489169
2020-07-10
53489170
2020-07-10
53489171
2020-07-10
53489172
2020-07-10
53489173
2020-07-10
53489174
2020-07-10
53489175
2020-07-10
53489176
2020-07-10
53489177
2020-07-10
53489178
2020-07-10
53489179
2020-07-10
53489180
2020-07-10
53489181
2020-07-10
53489182
2020-07-10
53489183
2020-07-10
53489184
2020-07-10
53489185
2020-07-10
53489186
2020-07-10
53489187
2020-07-10
53489188
2020-07-10
53489189
2020-07-10
53489190
2020-07-10
53489191
2020-07-10
53489192
2020-07-10
53489193
2020-07-10
53489194
2020-07-10
53489195
2020-07-10
53489196
2020-07-10
53489197
2020-07-10
53489198
2020-07-10
53489199
2020-07-10
53489200
2020-07-10
53489201
2020-07-10
53489202
2020-07-10
53489203
2020-07-10
53489204
2020-07-10
53489205
2020-07-10
53489206
2020-07-10
53489207
2020-07-10
53489208
2020-07-10
53489209
2020-07-10
53489210
2020-07-10
53489211
2020-07-10
53489212
2020-07-10
53489213
2020-07-10


In [133]:
word = pd.DataFrame(word)

In [161]:
words = []
for pos in np.argsort(-weight[0, :])[0:10]:
        print(pos)
        words.append(word[pos])

865
446
2122
85
788
3316
1940
1088
861
1766


In [154]:
import numpy as np
word.loc[np.argsort(-weight[0, :])]

,0
865,利用率
446,不锈钢
2122,板材
85,cypnest
788,兴化市
...,...
1171,商会
1172,商务
1173,商品
1162,哈姆雷特


In [130]:
np.argsort(weight[0, :])

array([   0, 2308, 2309, ...,   85,  446,  865])

In [67]:
ids[1]

'-2696153348591432670'

In [54]:
wdf

,id,keywords,weights
0,1525292380671331584,区域,9.2700
1,2955333838766622720,区域,18.5400
2,4735010527553998848,区域,9.2700
3,4068512270479018496,区域,6.9525
4,6688429756260698112,区域,9.2700
...,...,...,...
150,53489083,logserver,7.0754
151,53487906,轨道车,7.0754
152,53488097,teamview,7.0754
153,53487902,离谱,7.0754


In [46]:

doc['keywords'] = wdf[wdf.id == 6688429756260698112]['keywords'].tolist()

In [88]:
path = os.path.join(base_dir, ids[1] + '.txt')
with open(path, "r", encoding='utf-8') as f:
        title = f.readline().split()
        article = f.read()
segmentation(article.split())

['背景',
 '戴南指',
 '江苏省',
 '兴化市',
 '戴南镇',
 '区域',
 '不锈钢',
 '行业',
 '被誉为',
 '中国',
 '不锈钢',
 '名镇',
 '小作坊',
 '模式',
 '工人',
 '代加工',
 '压力',
 '商家',
 '利润',
 '板材',
 '利用率',
 '板材',
 '利用率',
 'CypNest',
 'CypNest',
 '痛点',
 '同行人',
 '代田田',
 '李鹏',
 '李春龙',
 '叶萍',
 '王新杰',
 '龙哥',
 '重点',
 '打印机',
 '会议纪要',
 '汇总',
 '感兴趣',
 '蓝屏',
 '田田',
 '电脑',
 '附图',
 '断面']

In [115]:
doc

{'id': '-8812005873635802417',
 'title': '出差报告【邵达】7月15-7月19日广州精密切割事宜出差报告见附件。',
 'text': '出差报告还得提高啊，至少整合到一起，排下版，图和文能对应起来，最后再来个总结，以及后续要做的工作哈\n',
 'url': 'http://oa.fscut.com/seeyon/bbs.do?method=bbsView&articleId=-8812005873635802417&spaceType=&spaceId='}

In [123]:
doc['keywords1'] = '报告'
doc['keywords2'] = '结论'

In [124]:
res = es.index(index="test", id=1, body=doc)
print(res['result'])
res = es.get(index="test", id=1)
print(res['_source'])

updated
{'id': '-8812005873635802417', 'title': '出差报告【邵达】7月15-7月19日广州精密切割事宜出差报告见附件。', 'text': '出差报告还得提高啊，至少整合到一起，排下版，图和文能对应起来，最后再来个总结，以及后续要做的工作哈\n', 'url': 'http://oa.fscut.com/seeyon/bbs.do?method=bbsView&articleId=-8812005873635802417&spaceType=&spaceId=', 'keywords': ['报告', '结论'], 'keywords1': '报告', 'keywords2': '结论'}
